In [6]:
import operator
import random
import pandas as pd

from deap import base, creator, tools, gp, algorithms

def make_psets():
    # ––––––––––––––––––––––––––––––––––––––––––––––––––
    # Conjunto F1 = {and, or, not}
    # ––––––––––––––––––––––––––––––––––––––––––––––––––
    pset1 = gp.PrimitiveSet("MAIN1", 3, prefix="X")
    pset1.addPrimitive(operator.and_, 2, name="AND")
    pset1.addPrimitive(operator.or_,  2, name="OR")
    pset1.addPrimitive(operator.not_, 1, name="NOT")
    pset1.addTerminal(0)
    pset1.addTerminal(1)
    # Opcional: renombrar las variables para que salgan como A, B, C
    pset1.renameArguments(ARG0="A", ARG1="B", ARG2="C")

    # ––––––––––––––––––––––––––––––––––––––––––––––––––
    # Conjunto F2 = F1 ∪ {xor}
    # ––––––––––––––––––––––––––––––––––––––––––––––––––
    pset2 = gp.PrimitiveSet("MAIN2", 3, prefix="X")
    # Repetimos manualmente las primitivas de F1
    pset2.addPrimitive(operator.and_, 2, name="AND")
    pset2.addPrimitive(operator.or_,  2, name="OR")
    pset2.addPrimitive(operator.not_, 1, name="NOT")
    # Añadimos XOR
    pset2.addPrimitive(lambda a, b: a ^ b, 2, name="XOR")
    pset2.addTerminal(0)
    pset2.addTerminal(1)
    pset2.renameArguments(ARG0="A", ARG1="B", ARG2="C")

    return pset1, pset2

# 3) Configuración de DEAP: fitness y tipo de individuo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

def setup_toolbox(pset):
    tb = base.Toolbox()
    tb.register("expr_init", gp.genFull, pset=pset, min_=1, max_=3)
    tb.register("individual", tools.initIterate, creator.Individual, tb.expr_init)
    tb.register("population", tools.initRepeat, list, tb.individual)
    tb.register("compile", gp.compile, pset=pset)

    # función de evaluación: contar coincidencias con la salida deseada
    def eval_parity(ind):
        f = tb.compile(expr=ind)
        return (sum(int(f(*inp)) == out for inp, out in zip(inputs, outputs)),)

    tb.register("evaluate", eval_parity)
    tb.register("select",    tools.selTournament, tournsize=3)
    tb.register("mate",      gp.cxOnePoint)
    tb.register("expr_mut",  gp.genFull, min_=0, max_=2)
    tb.register("mutate",    gp.mutUniform, expr=tb.expr_mut, pset=pset)

    # límites de tamaño para evitar explosión de árbol
    tb.decorate("mate",   gp.staticLimit(key=operator.attrgetter("height"), max_value=5))
    tb.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))
    return tb

# 4) Función que ejecuta GP para un PrimitiveSet dado
def run_gp(pset, ngen=40, pop_size=200):
    toolbox = setup_toolbox(pset)
    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)

    algorithms.eaSimple(pop, toolbox,
                        cxpb=0.5, mutpb=0.2,
                        ngen=ngen, halloffame=hof, verbose=False)

    best = hof[0]
    compile_best = toolbox.compile(expr=best)
    # exactitud final
    acc = sum(int(compile_best(*inp)) == out for inp, out in zip(inputs, outputs)) / len(inputs)
    return best, acc

if __name__ == "__main__":
    pset1, pset2 = make_psets()

    # Experimento con F1
    best1, acc1 = run_gp(pset1)
    print("Con F1 = {and, or, not}:")
    print(" Mejor individuo:", best1)
    print(f" Exactitud = {acc1*100:.1f}%\n")

    # Experimento con F2
    best2, acc2 = run_gp(pset2)
    print("Con F2 = {and, or, not, xor}:")
    print(" Mejor individuo:", best2)
    print(f" Exactitud = {acc2*100:.1f}%")



Con F1 = {and, or, not}:
 Mejor individuo: OR(OR(X2, OR(X2, AND(X1, X0))), NOT(OR(AND(X2, X1), OR(X0, OR(X2, X1)))))
 Exactitud = 75.0%

Con F2 = {and, or, not, xor}:
 Mejor individuo: XOR(AND(OR(OR(X1, X2), XOR(X0, 0)), NOT(XOR(XOR(1, X0), XOR(X1, X2)))), NOT(XOR(X1, X1)))
 Exactitud = 100.0%
